In [1]:
import nltk
import numpy as np
import pandas as pd

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import sklearn.metrics as m

In [2]:
import os

raw_texts_dir = "raw_texts"
data_list = []
label_list = []

# Read all the files in the raw_texts directory and store the data and labels in lists
for label in os.listdir(raw_texts_dir):
    label_dir = os.path.join(raw_texts_dir, label)
    if not os.path.isdir(label_dir):
        continue
    for filename in os.listdir(label_dir):
        if not filename.endswith(".txt"):
            continue
        filepath = os.path.join(label_dir, filename)
        with open(filepath, "r") as f:
            data = f.read()
            data_list.append(data)
            label_list.append(label)

len(data_list), len(label_list)

KeyboardInterrupt: 

In [ ]:

label_list = pd.DataFrame(label_list)
value1 = label_list[label_list[0] == "1"]
value2 = label_list[label_list[0] == "2"]
value3 = label_list[label_list[0] == "3"]
print("Olumlu Tweetler : ",len(value1))
print("Olumsuz Tweetler : ",len(value2))
print("Nötr Tweetler : ",len(value3))
print("Veri kümesindeki sınıf sayısı: ",len(label_list[0].unique()))
print("Veri kümesindeki toplam kayıt sayısı : ",len(label_list))

Olumlu Tweetler :  756
Olumsuz Tweetler :  1287
Nötr Tweetler :  957
Veri kümesindeki sınıf sayısı:  3
Veri kümesindeki toplam kayıt sayısı :  3000


In [ ]:
# download punkt package for tokenization
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\islam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# download stopwords package for removing stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\islam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
data_list = pd.DataFrame(data_list, columns=["text"])

In [ ]:
data_list.isnull().values.any()

False

In [ ]:
print(len(set(stopwords.words('turkish'))))

# Stopwords vocabulary
handmade_stopwords1 = open("stop_words/derlemtr2016-10000 (1).txt", encoding="utf-8").read().splitlines()[36:]

new_words = []
for word in handmade_stopwords1:
    new_word = ''.join(filter(lambda x: not x.isdigit() and not x.isspace(), word))
    new_words.append(new_word)

# new_words to txt file
with open("stop_words/derlemtr2016-10000 (1).txt", "w", encoding="utf-8") as f:
    for word in new_words:
        f.write(word + "\n")
        

# i changed because i just want to same variable name
handmade_stopwords1 = new_words
print(len(handmade_stopwords1))

# stopwords2
stopwords2 = open("stop_words/stop-words_turkish_1_tr.txt",encoding="utf-8").read().splitlines()[3:]
# concat two list
handmade_stopwords2 = stopwords2 + handmade_stopwords1
print(len(handmade_stopwords2))

# stopword3
stopwords3 = open("stop_words/stop-words_turkish_2_tr.txt",encoding="utf-8").read().splitlines()[3:]
# concat two list
handmade_stopwords3 = handmade_stopwords2 + stopwords3

print(len(handmade_stopwords3))

# concatenate all stopwords
handmade_stopwords = handmade_stopwords3 + list(set(stopwords.words('turkish')))
print(len(handmade_stopwords))

53
10002
10211
10322
10375


In [ ]:
"""
Birleştirdiğim stopwordsleri kullanmadım çünkü çok fazla kelime siliyordu ve accuracy düşüyordu. Bu yüzden sadece nltk stopwords kullandım. Denemek isterseniz 10. Hücredeki word=[stemmer.stem(i) for i in wordlist if i not in stopwords.words('turkish')] kodunu word=[stemmer.stem(i) for i in wordlist if i not in handmade_stopwords] olarak değiştirebilirsiniz.
"""

In [ ]:
print(data_list["text"][0])
import re

# Function to remove numbers
def remove_numeric(text):
    text = re.sub(r'\d+', '', text)
    return text

# pip install vngrs-nlp | https://vnlp.readthedocs.io/en/latest/main_classes/normalizer.html
from vnlp import Normalizer

normalizer = Normalizer()

# remove punctuation
def remove_punctuation(text):
    return normalizer.remove_punctuations(text)

# remove links like www. and http etc. and hashtags and usernames
def remove_links(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'www\S+', '', text)
    text = re.sub(r'@\S+', '', text)
    text = re.sub(r'#\S+', '', text)
    return text

# pip install Turkish-Deasciifier | https://github.com/emres/turkish-deasciifier
from turkish.deasciifier import Deasciifier

# Funtion to convert ascii characters to turkish characters
def deasciify(text):
    deasciifier = Deasciifier(text)
    my_deasciified_turkish_txt = deasciifier.convert_to_turkish()
    return my_deasciified_turkish_txt

data_list["text"] = data_list["text"].apply(remove_numeric)
data_list["text"] = data_list["text"].apply(remove_punctuation)
data_list["text"] = data_list["text"].apply(remove_links)
data_list["text"] = data_list["text"].apply(deasciify)

print(data_list["text"][0])


dun Turkcelle tepkilerimizden sonra bugün Turkcell twittera sponsor olmuş. Ne tesadüf değil mi ? :)
dün Türkcelle tepkilerimizden sonra bugün Türkcell twittera sponsor olmuş Ne tesadüf değil mi  


In [10]:
import re
# pip install zemberek-pyhon | https://github.com/loodos/zemberek-python
from zemberek import TurkishTokenizer
# pip install TurkishStemmer | https://github.com/otuncelli/turkish-stemmer-python
from TurkishStemmer import TurkishStemmer

stemmer = TurkishStemmer()

tokenizer = TurkishTokenizer.DEFAULT
content = []
sentences=[]
j=1

# First we tokenize the sentences and then we stem the words and remove the stopwords and then we join the words to make a sentence
for sen in data_list['text']:
  wordlist = []
  senti=sen.lower()
  words=tokenizer.tokenize(senti)
  for i in range(len(words)):
    wordlist.append(words[i].content)
  word=[stemmer.stem(i) for i in wordlist if i not in stopwords.words('turkish')]
  senti=' '.join(word)
  sentences.append(senti)
  print("Sentence ",j," stemmed and cleaned.")
  j=j+1

Sentence  1  stemmed and cleaned.
Sentence  2  stemmed and cleaned.
Sentence  3  stemmed and cleaned.
Sentence  4  stemmed and cleaned.
Sentence  5  stemmed and cleaned.
Sentence  6  stemmed and cleaned.
Sentence  7  stemmed and cleaned.
Sentence  8  stemmed and cleaned.
Sentence  9  stemmed and cleaned.
Sentence  10  stemmed and cleaned.
Sentence  11  stemmed and cleaned.
Sentence  12  stemmed and cleaned.
Sentence  13  stemmed and cleaned.
Sentence  14  stemmed and cleaned.
Sentence  15  stemmed and cleaned.
Sentence  16  stemmed and cleaned.
Sentence  17  stemmed and cleaned.
Sentence  18  stemmed and cleaned.
Sentence  19  stemmed and cleaned.
Sentence  20  stemmed and cleaned.
Sentence  21  stemmed and cleaned.
Sentence  22  stemmed and cleaned.
Sentence  23  stemmed and cleaned.
Sentence  24  stemmed and cleaned.
Sentence  25  stemmed and cleaned.
Sentence  26  stemmed and cleaned.
Sentence  27  stemmed and cleaned.
Sentence  28  stemmed and cleaned.
Sentence  29  stemmed and cle

In [11]:
sentences

['dün türkcel tepki sonr bugü türkcell twittera sponsor ol tesadüf değil mi',
 'girmez türkcell kart bil var yar itibaren on takıca twittersiz dene sade gün yapabil',
 'tam tünel geçer g çekiyordud türkcell reklam vodafone hareke geçir olma',
 'türkcell superonlin fiber internet adsl sabit telefon hizmet',
 'benç gnçtrkçll ark winterfest olur gelir',
 'türkcell san kıyak geçiyor d',
 'türkcel bağlan haya diyor',
 'kims takmıyo ben yaaaa türkcell ban mesaj aaat d',
 'türkcell pusu uygulama iphone kullanıcı büyük nimet oturduk yer nöbetç eczane kaç metr ötem ban söylüyor',
 'zaman sor yok ben türkcell iyi bir tarif arıyor yap karar vere fikir versen',
 'samsung türkcell kaçırılmayacak kampanya samsung an bağlantı kalma sağlayan qwerty klavye',
 'i̇novasyon ödül türkcell',
 'pazar gün şans günümmüss türkcel bedav konuscakmışım sağol varol türkcell kahrol avea',
 'büyük ödül türkcell teknoloji',
 'hadi yin süper iyi sıyır i̇phone s olay',
 'superonlin fiber internet hizm adres var tavsi ed

In [12]:
# tf-idf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
tfidf=TfidfVectorizer(max_features=10000)
     
# Calculate tf-idf values
features=tfidf.fit_transform(sentences)
     

# Get the feature names
feature_names = tfidf.get_feature_names_out()

# Transform the tf-idf values to a dataframe
tfidf_df = pd.DataFrame(features.toarray(), columns=feature_names)

In [15]:
tfidf_df["Sınıf"] = label_list

In [16]:
tfidf_df

,aa,aaa,aaaaaooooo,aaaahhh,aaat,aahh,aasdfghjksasdfghj,abart,abartmayal,abartmışbilinçalt,...,şör,şörün,şöyl,şöyma,şükür,şüph,şüphelenme,şımartiyo,şıpesil,Sınıf
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


In [36]:
# to txt
tfidf_df.to_csv("tfidf_data.txt", index=True) 
#tfidf_df.to_csv("tfidf_data.csv", index=True)

In [18]:
len(tfidf.get_feature_names_out())

7433

In [19]:
# avoiding warning
label_list = np.ravel(label_list)

In [20]:
feature_train,feature_test,label_train,label_test=train_test_split(features,label_list,test_size=0.2,random_state=42)

In [21]:
feature_train.shape, feature_test.shape, label_train.shape, label_test.shape

((2400, 7433), (600, 7433), (2400,), (600,))

In [22]:
# Multinomial Naive Bayes
model=MultinomialNB()
model.fit(feature_train,label_train)

MultinomialNB()

In [23]:
label_pred=model.predict(feature_test)

In [24]:
label_pred

array(['2', '2', '2', '1', '2', '2', '2', '2', '1', '2', '2', '1', '2',
       '2', '3', '2', '2', '2', '3', '2', '2', '2', '2', '3', '2', '3',
       '2', '2', '3', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2',
       '2', '1', '2', '2', '1', '2', '2', '2', '3', '2', '2', '2', '2',
       '1', '3', '1', '2', '2', '2', '2', '3', '3', '2', '2', '2', '1',
       '2', '3', '2', '2', '2', '2', '2', '3', '2', '2', '3', '2', '2',
       '1', '3', '2', '1', '2', '2', '2', '2', '2', '2', '2', '2', '1',
       '2', '2', '1', '2', '2', '3', '3', '1', '2', '2', '2', '2', '1',
       '2', '2', '2', '2', '2', '2', '2', '3', '2', '2', '1', '3', '2',
       '3', '2', '3', '2', '2', '2', '2', '2', '1', '3', '2', '2', '2',
       '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2',
       '3', '2', '2', '3', '2', '2', '2', '2', '2', '2', '1', '1', '2',
       '2', '2', '2', '3', '2', '2', '2', '2', '2', '2', '2', '3', '2',
       '1', '2', '2', '2', '2', '3', '2', '3', '3', '2', '2', '2

In [25]:
label_test

array(['2', '2', '2', '1', '3', '2', '2', '3', '1', '3', '3', '1', '1',
       '2', '3', '2', '2', '1', '3', '3', '1', '1', '2', '3', '2', '3',
       '3', '2', '1', '2', '2', '2', '1', '3', '1', '2', '3', '1', '1',
       '2', '1', '1', '1', '1', '2', '2', '3', '1', '3', '2', '2', '3',
       '1', '3', '1', '3', '2', '2', '3', '2', '1', '1', '2', '1', '1',
       '2', '2', '2', '2', '1', '2', '3', '3', '2', '1', '1', '3', '3',
       '3', '3', '2', '2', '2', '2', '2', '1', '3', '2', '3', '3', '1',
       '2', '3', '2', '2', '1', '3', '3', '1', '2', '3', '1', '1', '2',
       '2', '2', '2', '1', '1', '2', '3', '1', '3', '3', '1', '3', '1',
       '1', '2', '2', '3', '2', '1', '2', '2', '3', '2', '2', '2', '2',
       '3', '2', '2', '2', '2', '1', '1', '2', '2', '3', '3', '2', '3',
       '3', '3', '1', '3', '3', '2', '1', '2', '3', '2', '3', '1', '2',
       '1', '3', '2', '3', '2', '2', '1', '1', '2', '2', '1', '3', '3',
       '1', '3', '3', '2', '2', '3', '2', '3', '2', '1', '2', '1

In [26]:
m.accuracy_score(label_test,label_pred)

0.5533333333333333

0.5216666666666666

In [28]:
print(m.classification_report(label_test,label_pred))

              precision    recall  f1-score   support

           1       0.78      0.25      0.38       166
           2       0.52      0.92      0.67       256
           3       0.57      0.30      0.40       178

    accuracy                           0.55       600
   macro avg       0.62      0.49      0.48       600
weighted avg       0.61      0.55      0.51       600



In [29]:
print(m.confusion_matrix(label_test,label_pred))

[[ 42  97  27]
 [  6 236  14]
 [  6 118  54]]


In [30]:
len(label_test)

600

In [31]:
# stratified k-fold cross validation
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

# k-fold cross validation
kfold = StratifiedKFold(n_splits=10, random_state=7, shuffle=True)
results = cross_val_score(model, features, label_list, cv=kfold)

# Precision for each class
precisions = m.precision_score(label_test, label_pred, average=None)
recalls = m.recall_score(label_test, label_pred, average=None)
f1s = m.f1_score(label_test, label_pred, average=None)

# calculate micro and macro average
micro_precision = m.precision_score(label_test, label_pred, average="micro")
micro_recall = m.recall_score(label_test, label_pred, average="micro")
micro_f1 = m.f1_score(label_test, label_pred, average="micro")

macro_precision = m.precision_score(label_test, label_pred, average="macro")
macro_recall = m.recall_score(label_test, label_pred, average="macro")
macro_f1 = m.f1_score(label_test, label_pred, average="macro")

# calculate TP, FP, FN for each class
TP = []
FP = []
FN = []

for i in range(3):
    TP.append(m.confusion_matrix(label_test, label_pred)[i][i])
    FP.append(sum(m.confusion_matrix(label_test, label_pred)[i]) - TP[i])
    FN.append(sum(m.confusion_matrix(label_test, label_pred)[:,i]) - TP[i])



# create a dataframe to show the results
cross_validation={
    " ":["Precision","Recall","F-Score","TP adedi","FP adedi","FN adedi"],
    "Sınıf 1 (olumlu) ": [precisions[0],recalls[0],f1s[0],TP[0],FP[0],FN[0]],
    "Sınıf 2 (olumsuz) ": [precisions[1],recalls[1],f1s[1],TP[1],FP[1],FN[1]],
    "Sınıf 3 (nötr)": [precisions[2],recalls[2],f1s[2],TP[2],FP[2],FN[2]],
    "MACRO avg.":[macro_precision,macro_recall,macro_f1,"-","-","-"],
    "Micro avg.": [micro_precision,micro_recall,micro_f1,"-","-","-"]
}


In [32]:
len(label_test)

600

In [33]:
df_cross_validation = pd.DataFrame(cross_validation)
print(df_cross_validation)

              Sınıf 1 (olumlu)   Sınıf 2 (olumsuz)   Sınıf 3 (nötr)  \
0  Precision           0.777778            0.523282        0.568421   
1     Recall           0.253012            0.921875        0.303371   
2    F-Score           0.381818            0.667610        0.395604   
3   TP adedi          42.000000          236.000000       54.000000   
4   FP adedi         124.000000           20.000000      124.000000   
5   FN adedi          12.000000          215.000000       41.000000   

  MACRO avg. Micro avg.  
0    0.62316   0.553333  
1   0.492753   0.553333  
2   0.481677   0.553333  
3          -          -  
4          -          -  
5          -          -  


In [35]:
# to csv
df_cross_validation.to_csv("df_cross_validation.csv", index=False)

In [35]:
probs = model.feature_log_prob_
probs

array([[-8.89866918, -8.89866918, -8.89866918, ..., -8.89866918,
        -8.34072487, -8.89866918],
       [-8.35076511, -9.03040393, -8.70421081, ..., -9.03040393,
        -9.03040393, -9.03040393],
       [-8.58381959, -8.67445009, -8.95091689, ..., -8.95091689,
        -8.95091689, -8.49761929]])

In [36]:
model.get_params

<bound method BaseEstimator.get_params of MultinomialNB()>

In [37]:

tfidf.get_feature_names_out()

array(['aa', 'aaa', 'aaaaaooooo', ..., 'şüphelenme', 'şımartiyo',
       'şıpesil'], dtype=object)

In [38]:
# sort the probabilities and get the words
sorted(zip(model.feature_log_prob_[0],tfidf.get_feature_names_out()),reverse=True)

[(-4.908223012757519, 'türkcell'),
 (-5.401801593074003, 'paylaş'),
 (-5.636056759599341, 'superonlin'),
 (-6.468633344045234, 'reklam'),
 (-6.651648619902321, 'türkcel'),
 (-6.685127139094424, 'fiber'),
 (-6.801020986687886, 'avea'),
 (-7.017384815742775, 'gnçtrkçll'),
 (-7.034039134482655, 'vodafone'),
 (-7.03961568189401, 'bağlan'),
 (-7.059364169629754, 'ttnet'),
 (-7.118046142944549, 'öztürkcell'),
 (-7.18450133316845, 'novasyonda'),
 (-7.253336554900964, 'sms'),
 (-7.301718115680149, 'selocan'),
 (-7.303250549094587, 'vall'),
 (-7.325701965416506, 'memnu'),
 (-7.453831049440496, 'paylaşinca'),
 (-7.458628415597175, 'türkcellmuzik'),
 (-7.4699294400013585, 'pazarlamakonferans'),
 (-7.485175316894294, 'kank'),
 (-7.485819368989481, 'laa'),
 (-7.505480365183352, 'internet'),
 (-7.531807980506777, 'yi'),
 (-7.5432176137957425, 'türkiye'),
 (-7.545513909174867, 'kampanya'),
 (-7.545762902157753, 'operatör'),
 (-7.556197400004042, 'kıyak'),
 (-7.567989400548289, 'ınternet'),
 (-7.57074